# Binary shapes in 2D
### Simulation written by Rachael Skye and Maya Martirossyan

In [ ]:
!tar -xzf ./hoomd-v2.9.7.tar.gz
#!pip install pip-tools
#!pip-compile requirements.txt

In [ ]:
# importing modules

import hoomd
import hoomd.hpmc as hpmc
import coxeter
import ipywidgets as widgets
import gsd, gsd.hoomd
import plato.draw.vispy as draw
import numpy as np
import matplotlib.pyplot as pp
import random
#%gui qt

### Setup Functions

Setting up some functions that we will use throughout our simulation code. These need to be evaluated, but do not need to be interacted with. 

In [ ]:
# Functions for defining shapes

get_3D_Shape = coxeter.families.RegularNGonFamily()

def make3D(vertex_list):
    """
    input: vertex_list for 2D shapes
    output: vertex_list for same shapes but "in 3D"
    """
    to_add = np.transpose([np.zeros(len(vertex_list))])
    vertex_list = np.concatenate([vertex_list,to_add],axis=1)
    return vertex_list

def get_shape(vertex_count):
    return make3D(get_3D_Shape.make_vertices(vertex_count))

#the interactive display for selecting fraction of each shape
shape_ratio = widgets.FloatSlider(
    min=0.00, 
    max=1.00, 
    step=0.01, 
    description='Shape ratio (%A):', 
    value=0.50,
    readout_format='.2f')

#Colors of each shape
# setting colors
# RGB values between 0 and 1
color_A = np.float32(np.divide([255, 50, 150], 255)) #pink
color_B = np.float32(np.divide([0, 200, 100], 255)) #green

In [ ]:
# Functions for initializing the simulation with HOOMD

def initialize(num_particles): 
    init = hoomd.init.create_lattice(unitcell=hoomd.lattice.unitcell(N=2,
                                                                  a1=[3,0,0],
                                                                  a2=[0,6,0],
                                                                  a3=[0,0,1],
                                                                  dimensions=2,
                                                                  position=[[0,0,0],
                                                                            [0,3,0]],
                                                                  type_name=["A","B"]),
                                n=[num_particles,int(1/2*num_particles)])
    return(init)
    
    
def set_shape_ratio(system, ratio):
    # taking snapshot of the first frame
    snap = system.take_snapshot()

    # defining particle types
    snap.particles.types = ["A","B"]

    nParticles = len(snap.particles.typeid)
    
    # first, assign all particles to be type B
    for p in range(nParticles):
        snap.particles.typeid[p] = 1
    
    # next, randomly assign certain particles to be type A
    partnum_A = int(nParticles*ratio.value)
    idlistA=[]
    for p in range(partnum_A):
        r = random.randint(0, nParticles - 1)
        while r in idlistA:
            r = random.randint(0, nParticles - 1)
        snap.particles.typeid[r] = 0
        idlistA.append(r)
        
    return snap

In [ ]:
# Functions for visualizing our simulation

def draw_2D(snap, verticesA, verticesB):
    # primA and primB define our two groups of particles (A type and B type)
    primA = draw.Polygons(vertices = verticesA[:,:2])
    primB = draw.Polygons(vertices = verticesB[:,:2])

    # drawing the box
    box_prim = draw.Box(color=(0, 0, 0, 1), width=.2) 

    # drawing the scene with both primitives
    scene = draw.Scene([primA, box_prim], zoom=.5, clip_scale=5)
    scene.add_primitive(primB)
    scene.show()

    box = [snap.box.Lx, snap.box.Ly, snap.box.Lz, snap.box.xy, snap.box.xz, snap.box.yz]
    for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
        setattr(box_prim, name, val)

    # reading in particle positions
    primA.positions = snap.particles.position[snap.particles.typeid == 0,:2]
    primB.positions = snap.particles.position[snap.particles.typeid == 1,:2]

    # reading in shape orientations
    primA.orientations = snap.particles.orientation
    primB.orientations = snap.particles.orientation

    # reading in shape sizes
    primA.diameters = np.full(len(snap.particles.position), 1)
    primB.diameters = np.full(len(snap.particles.position), 1)

    # colorsA = RGBA from 0 to 1
    colorsA = np.ones((len(primA.positions), 4))
    colorsA[:, :3] = color_A
    primA.colors = colorsA

    # colorsB = RGBA from 0 to 1
    colorsB = np.ones((len(primB.positions), 4))
    colorsB[:, :3] = color_B
    primB.colors = colorsB

    scene.render()
    
    
def getFrameCount(fname):
    """
    inputs: fname, the filename (ex: 'dump.gsd')
    outputs: len(traj), number of frames in simulation
    """
    with gsd.hoomd.open(fname, 'rb') as traj:
        return len(traj)

### Choosing parameters for our simulation

This includes choosing the two shapes we will be using, the ratio between the shapes, and the total number of shapes in the simulation box. Here, user input is important! 

As a scientist, you will want to also make a prediction (hypothesis) at this point. Think about how the simulation outcome might depend on your choice of shapes and their ratio. A good experiment also only changes one thing at a time -- decide which parameter you'd like to vary and which you'd like to keep constant.

In [ ]:
# we will be using regular polygons for our shapes

# choose shapes by the number of vertices that shape has
# Example: verticesA = get_shape(3) will choose triangles for shapeA
verticesA = get_shape(3)
verticesB = get_shape(8)

In [ ]:
# choose a fraction between 0 and 1
# which will represent the percent of shapes that are shape A
# example:
# shape_ratio = 0 means all the shapes will be shape B

display(shape_ratio)

In [ ]:
# setting number of particles along one side of our simulation box
# which is the square root of the total number of particles

num_particles_sqrt = 20

### Initializing the simulation

Here, we will use the functions for simulation initialization written above, and execute them with the inputs given by the user. We can then visualize what we've initialized -- the first frame of the simulation.

In [ ]:
hoomd.context.initialize("--mode=cpu");

# defining the first frame of our simulation
# set up lattice of particles
system = initialize(num_particles_sqrt)

#Set the system to have the correct shape ratio
system_with_shapes = set_shape_ratio(system, shape_ratio)
    
system.restore_snapshot(system_with_shapes)

In [ ]:
# visualizing our initial frame with plato

initial_frame = system.take_snapshot()

draw_2D(initial_frame, verticesA, verticesB)

### Running the simulation

Now, we finally execute the simulation itself. This has a couple steps -- first, we set some basic parameters, such as timesteps, pressure, and how 'hard' we are shaking our simulation box (more technically, how large our steps/rotations are). In the next cell, we record the data in the form of a gsd file (with the simulation frames) and a log file (with some simulation parameters that help us see how volume and pressure are changing).

Note: If you have already run a simulation, make sure to update the filename that your new simulation will be saved to (in the second cell below).

In [ ]:
# simulation setup

# pressure and time settings
total_steps = 2e5
max_pressure = 10

# Monte Carlo settings
mc = hpmc.integrate.convex_polygon(seed=np.random.randint(1,1e6), d=0.5, a=0.5, move_ratio=0.5)
mc.shape_param.set('A', vertices=verticesA)
mc.shape_param.set('B', vertices=verticesB)

boxMC = hpmc.update.boxmc(mc, hoomd.variant.linear_interp(points=[[0,1],[total_steps,max_pressure]],zero=0), 
                          np.random.randint(1,1e6))

boxMC.volume(delta=1.0, weight=1.0)

# define our simulation tuners
particle_tuner = hpmc.util.tune(obj=mc, tunables=['d','a'], target=0.2)
box_tuner = hpmc.util.tune_npt(obj=boxMC, tunables=['dV'],  target=0.2)

def update_tuner(particle_tuner, box_tuner):
    particle_tuner.update()
    box_tuner.update()


all=hoomd.group.all()

In [ ]:
# record data

# for visualizations
traj = hoomd.dump.gsd(filename='traj.gsd',period=int(total_steps/100),group=all,overwrite=True)
traj.dump_shape(mc)

# for plots
logger = hoomd.analyze.log(filename='log.txt',
    quantities=['hpmc_translate_acceptance',
                'hpmc_rotate_acceptance',
                'hpmc_boxmc_volume_acceptance',
                'hpmc_d',
                'lx',
                'hpmc_boxmc_betaP',
                'hpmc_overlap_count'
                ],
    period=int(total_steps/100),
    overwrite=True)

In [ ]:
# running simulation

# defining number of times we will tune the step size parameters
number_loops = 100
loop_steps = int(total_steps/number_loops)

hoomd.run(total_steps, 
          callback_period = loop_steps, 
          callback = lambda tuner: update_tuner(particle_tuner,box_tuner))

### Visualizing your simulation

Your simulation is now complete! It's time to see what the result was.

In [ ]:
filename = 'traj.gsd'
frame_num = getFrameCount(filename)

In [ ]:
primA = draw.Polygons(vertices = verticesA[:,:2])
primB = draw.Polygons(vertices = verticesB[:,:2])
box_prim = draw.Box(color=(0, 0, 0, 1), width=.2)

scene = draw.Scene([primA, box_prim], zoom=.5, clip_scale=5)
scene.add_primitive(primB)
scene.show()
            

# visualizations
# looping over frames
@widgets.interact(frame_index=(0, frame_num-1, 1))
def plot(frame_index=0):
    with gsd.hoomd.open(filename, 'rb') as traj:
        
        frame = traj[frame_index]
        box = frame.configuration.box
        for (name, val) in zip(['Lx', 'Ly', 'Lz', 'xy', 'xz', 'yz'], box):
            setattr(box_prim, name, val)

        primA.positions = frame.particles.position[frame.particles.typeid == 0,:2]
        primB.positions = frame.particles.position[frame.particles.typeid == 1,:2]
        
        primA.orientations = frame.particles.orientation
        primB.orientations = frame.particles.orientation
        
        primA.diameters = np.full(len(frame.particles.position), 1)
        primB.diameters = np.full(len(frame.particles.position), 1)
        
        colorsA = np.ones((len(primA.positions), 4))
        colorsA[:, :3] = color_A
        primA.colors = colorsA
        
        colorsB = np.ones((len(primB.positions), 4))
        colorsB[:, :3] = color_B
        primB.colors = colorsB
    scene.render()

### Plotting simulation data over time

In [ ]:
data = np.loadtxt('log.txt',skiprows=1)

In [ ]:
# Volume changes with time

time = data[:,0]
lx = data[:,5]
volume = lx ** 3

pp.scatter(time, volume);
pp.title('Volume change over timesteps');
pp.xlabel('Steps');
pp.ylabel('Volume');

In [ ]:
# Volume changes with pressure

pressure = data[:,6]
lx = data[:,5]
volume = lx ** 2

pp.scatter(pressure, volume);
pp.title('Volume change over pressure');
pp.xlabel('Pressure');
pp.ylabel('Volume');

In [ ]:
# Zoom in on the part below the initial compression

pp.scatter(pressure, volume);
pp.title('Volume change over pressure');
pp.xlabel('Pressure');
pp.ylabel('Volume');

pp.ylim(400,1000)